<a href="https://colab.research.google.com/github/ShyamSundhar1411/My-ML-Notebooks/blob/master/Transfer%20Learning/Transfer_Learning_in_TensorFlow_Feature_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 ## Introduction to Transfer Learning - Feature Extraction

In [1]:
!nvidia-smi

Thu Jan 12 18:03:45 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## 1. Downloading and becoming one with data

In [2]:
import zipfile
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

--2023-01-12 18:03:45--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.115.128, 172.253.122.128, 172.253.63.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.115.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M   188MB/s    in 0.9s    

2023-01-12 18:03:46 (188 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [3]:
zip_ref = zipfile.ZipFile("10_food_classes_10_percent.zip")
zip_ref.extractall()
zip_ref.close()

In [4]:
import os

for dirpath,dirnames,filenames in os.walk("10_food_classes_10_percent"):
  print("There are ",len(dirnames)," directories",len(filenames)," in ",dirpath)

There are  2  directories 0  in  10_food_classes_10_percent
There are  10  directories 0  in  10_food_classes_10_percent/train
There are  0  directories 75  in  10_food_classes_10_percent/train/pizza
There are  0  directories 75  in  10_food_classes_10_percent/train/ramen
There are  0  directories 75  in  10_food_classes_10_percent/train/ice_cream
There are  0  directories 75  in  10_food_classes_10_percent/train/sushi
There are  0  directories 75  in  10_food_classes_10_percent/train/chicken_curry
There are  0  directories 75  in  10_food_classes_10_percent/train/hamburger
There are  0  directories 75  in  10_food_classes_10_percent/train/grilled_salmon
There are  0  directories 75  in  10_food_classes_10_percent/train/chicken_wings
There are  0  directories 75  in  10_food_classes_10_percent/train/fried_rice
There are  0  directories 75  in  10_food_classes_10_percent/train/steak
There are  10  directories 0  in  10_food_classes_10_percent/test
There are  0  directories 250  in  10_f

## 2. Creating data loaders

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SHAPE = (224,224)
BATCH_SIZE = 32
EPOCHS = 5
train_dir = "10_food_classes_10_percent/train/"
test_dir = "10_food_classes_10_percent/test/"
train_datagen = ImageDataGenerator(rescale = 1/255.)
test_datagen = ImageDataGenerator(rescale = 1/255.)

In [6]:
train_data = train_datagen.flow_from_directory(train_dir,target_size = IMAGE_SHAPE,batch_size = BATCH_SIZE,class_mode = "sparse")

Found 750 images belonging to 10 classes.


In [7]:
test_data = test_datagen.flow_from_directory(test_dir,target_size = IMAGE_SHAPE,batch_size = BATCH_SIZE,class_mode = "sparse")

Found 2500 images belonging to 10 classes.


## 3.Setting Up Callbacks

### Tensorboard Callback

In [8]:
 ## Creating tensorboard callback
import datetime

def create_tensorboard_callback(dir_name,experiment_name):
  log_dir = dir_name + "/"+experiment_name+"/"+datetime.datetime.now().strftime("%Y%m%D-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir)
  print("Saving Tensorflow log files to: ",log_dir)
  return tensorboard_callback


## Creating Models Using TensorFlow Hub

In [13]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers

In [14]:
resnet_url = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4"
efficient_url = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"

In [26]:
def create_model(model_url,max_classes = 10):
  """
  Takes a tensorflow hub url and creates a keras sequential model with it.
  """
  feature_extractor_layer = hub.KerasLayer(model_url,trainable = False,name = "feature_extraction_layer",input_shape = IMAGE_SHAPE+(3,))
  model = tf.keras.Sequential([
      feature_extractor_layer,
      tf.keras.layers.Dense(max_classes,activation = "softmax",name = "output_layer")
  ])
  return model

In [27]:
resnet_model = create_model(resnet_url,10)

In [28]:
resnet_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 feature_extraction_layer (K  (None, 2048)             23564800  
 erasLayer)                                                      
                                                                 
 output_layer (Dense)        (None, 10)                20490     
                                                                 
Total params: 23,585,290
Trainable params: 20,490
Non-trainable params: 23,564,800
_________________________________________________________________


In [29]:
resnet_model.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(),optimizer = "adam",metrics = ["accuracy"])

In [ ]:
history_1 = resnet_model.fit(train_data,steps_per_epoch = len(train_data),epochs = 5 ,validation_data = test_data,validation_steps = len(test_data),callbacks = [create_tensorboard_callback(dir_name = "tensorflow_hub",experiment_name = "resnet_v2_50")])

Saving Tensorflow log files to:  tensorflow_hub/resnet_v2_50/20230101/12/23-181915
Epoch 1/5
24/24 [==============================] - 17s 704ms/step - loss: 0.0737 - accuracy: 0.9973 - val_loss: 0.6100 - val_accuracy: 0.7968
Epoch 2/5
24/24 [==============================] - ETA: 0s - loss: 0.0672 - accuracy: 0.9987